In [ ]:
#| default_exp basics

In [ ]:
#| export
from id_fastcore.imports import *
import builtins, types
import pprint
try: from types import Union
except ImportError: Union = None

In [ ]:
#| hide
from __future__ import annotations
from fastcore.test import *
from nbdev.showdoc import *
from fastcore.nb_imports import *

# Basic functionality

>Basic functionality used in the fastai library

In [ ]:
#| export
defaults = SimpleNamespace()

In [ ]:
#| hide
defaults.attr1 = 'Mary'
defaults.attr2 = 23

defaults

namespace(attr1='Mary', attr2=23)

In [ ]:
#| export
def ifnone(a, b):
    "`b` if `a` is None else `a`"
    return b if a is None else a

Since `b if a is None else a` is such a common pattern, we wrap it in a function. However, be careful, because python will evaluate both `a` and `b` when calling `ifnone` (which it doesn't do if using the `if` version directly).

In [ ]:
test_eq(ifnone(None,1), 1)
test_eq(ifnone(2,   1), 2)

In [ ]:
#| export
def maybe_attr(o, attr):
    "`getattr(o,attr,o)`"
    return getattr(o, attr, o)

Return the attribute `attr` for object `o`. If the attribute doesn't exist, then return the object `o` instead.

In [ ]:
class myobj: myattr='foo'

test_eq(maybe_attr(myobj, 'myattr'), 'foo')
test_eq(maybe_attr(myobj, 'another_attr'), myobj)     

In [ ]:
#| export
def basic_repr(flds=None):
    "Minimal `__repr__`"
    if isinstance(flds, str): flds = re.split(', *', flds)
    flds = list(flds or [])
    def _f(self):
        res = f'{type(self).__module__}.{type(self).__name__}'
        if not flds: return f'<{res}>'
        sig = ', '.join(f'{o} = {getattr(self, o)!r}' for o in flds)
        return f'{res}({sig})'
    return _f

In types which provide rich display functionality in Jupyter, their `__repr__` is also called in order to provide a fallback text representation. Unfortunately, this includes a memory address which changes on every invocation, making it non-deterministic. This causes diffs to get messy and creates conflicts in git. To fix this, put `__repr__=basic_repr()` inside your class.

In [ ]:
class SomeClass: __repr__=basic_repr()
repr(SomeClass())

'<__main__.SomeClass>'

If you pass a list of attributes (`flds`) of an object, then this will generate a string with the name of each attribute and its corresponding value. The format of this string is key=value, where key is the name of the attribute, and value is the value of the attribute. For each value, attempt to use the `__name__` attribute with the value's `__repr__` when constructing the string.

In [ ]:
class SomeClass:
    a=1
    b='foo'
    __repr__=basic_repr('a,b')
    __name__='some-class'

repr(SomeClass())
     

"__main__.SomeClass(a = 1, b = 'foo')"

In [ ]:
class AnotherClass:
    c=SomeClass()
    d='bar'
    __repr__=basic_repr(['c', 'd'])

repr(AnotherClass())

"__main__.AnotherClass(c = __main__.SomeClass(a = 1, b = 'foo'), d = 'bar')"

In [ ]:
#| export
def is_array(x):
    "`True` if `x` supports `__array__` or `iloc`"
    return hasattr(x, '__array__') or hasattr(x, 'iloc')

In [ ]:
is_array(np.array(1)), is_array([1])

(True, False)

In [ ]:
#| export
def listify(o=None, *rest, use_list=False, match=None):
    "Convert `o` to a `list`"
    if rest: o = (o,) + rest
    # if use_list - shortcut to [o]
    if use_list: res = list(o)
    elif o is None: res = []
    elif isinstance(o, list): res = o
    elif isinstance(o, str) or is_array(o): res = [o]
    elif is_iter(o): res = list(o)
    else: res = [o]
    if match is not None:
        if is_coll(match): match = len(match)
        if len(res) == 1: res = match * res
        else: assert len(res)==match, 'Match length mismatch'
    return res

Conversion is designed to "do what you mean", e.g:

In [ ]:
test_eq(listify('hi'), ['hi'])
test_eq(listify(array(1)), [array(1)])
test_eq(listify(1), [1])
test_eq(listify([1,2]), [1,2])
test_eq(listify(range(3)), [0,1,2])
test_eq(listify(None), [])
test_eq(listify(1,2), [1,2])

In [ ]:
arr = np.arange(9).reshape(3,3)
listify(arr)

[array([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])]

In [ ]:
listify(array([1,2]))

[array([1, 2])]

Generators are turned into lists too:

In [ ]:
gen = (o for o in range(3))
test_eq(listify(gen), [0,1,2])

Use `match` to provide a length to match:

In [ ]:
test_eq(listify(1,match=3), [1,1,1])

If `match` is a sequence, it's length is used:

In [ ]:
test_eq(listify(1,match=range(3)), [1,1,1])

If the listified item is not of length `1`, it must be the same length as `match`:

In [ ]:
test_eq(listify([1,1,1],match=3), [1,1,1])
test_fail(lambda: listify([1,1],match=3))

In [ ]:
#|export
def tuplify(o, use_list=False, match=None):
    "Make `o` a tuple"
    return tuple(listify(o, use_list=use_list, match=match))

In [ ]:
test_eq(tuplify(None),())
test_eq(tuplify([1,2,3]),(1,2,3))
test_eq(tuplify(1,match=[1,2,3]),(1,1,1))

In [ ]:
#|export
def true(x):
    "Test whether `x` is truthy; collections with >0 elements are considered `True`"
    try: return bool(len(x))
    except: return bool(x)

In [ ]:
[(o,true(o)) for o in
 (array(0),array(1),array([0]),array([0,1]),1,0,'',None)]

[(array(0), False),
 (array(1), True),
 (array([0]), True),
 (array([0, 1]), True),
 (1, True),
 (0, False),
 ('', False),
 (None, False)]

In [ ]:
#|export
class NullType:
    "An object that is `False` and can be called, chained, and indexed"
    def __getattr__(self, *args): return null
    def __call__(self, *args, **kwargs): return null
    def __getitem__(self, *args): return null
    def __bool__(self): return False

null = NullType()

In [ ]:
bool(null.hi().there[3])

False

In [ ]:
#|export
def tonull(x):
    "Convert `None` to `null`"
    return null if x is None else x

In [ ]:
bool(tonull(None).hi().there[3])

False

In [ ]:
#|export
def get_class(nm, *fld_names, sup=None, doc=None, funcs=None, **flds):
    "Dynamically create a class, optionally inheriting from `sup`, containing `fld_names`"
    attrs = {}
    for f in fld_names: attrs[f] = None
    for f in listify(funcs): attrs[f.__name__] = f
    for k,v in flds.items(): attrs[k] = v
    sup = ifnone(sup,())
    # because type accepts second argument as a tuple
    if not isinstance(sup, tuple): sup = (sup,)
    
    def _init(self, *args, **kwargs):
        # sets attrs for any kwargs, and for any args (matching by position to fields)
        for i,v in enumerate(args): setattr(self, list(attrs.keys())[i], v)
        # additional to get_class kwargs can be passed here
        for k,v in kwargs.items(): setattr(self,k,v)
        
    all_flds = [*fld_names, *flds.keys()]
    
    def _eq(self, b):
        return all([getattr(self,k) == getattr(b, k) for k in all_flds])
    
    if not sup: attrs['__repr__'] = basic_repr(all_flds)
    attrs['__init__'] = _init
    attrs['__eq__'] = _eq
    # res is created as a class with name nm, base class and attrs
    res = type(nm, sup, attrs)
    if doc is not None: res.__doc__ = doc
    return res

In [ ]:
show_doc(get_class)

---

[source](https://github.com/Iamalos/id-fastcore/blob/main/id_fastcore/basics.py#L88){target="_blank" style="float:right; font-size:smaller"}

### get_class

>      get_class (nm, *fld_names, sup=None, doc=None, funcs=None, **flds)

Dynamically create a class, optionally inheriting from `sup`, containing `fld_names`

In [ ]:
_t = get_class('_t', 'a', b=2)
t = _t()
test_eq(t.a, None)
test_eq(t.b, 2)
t = _t(1, b=3)
test_eq(t.a, 1)
test_eq(t.b, 3)
t = _t(1, 3)
test_eq(t.a, 1)
test_eq(t.b, 3)
test_eq(t, pickle.loads(pickle.dumps(t)))
repr(t)

'__main__._t(a = 1, b = 3)'

In [ ]:
_t = get_class('_t', 'a', 'b', 'c', d=2)
t = _t(1,2,d=3)
test_eq(t.a, 1)
test_eq(t.b, 2)
test_eq(t.c, None)

Most often you'll want to call `mk_class`, since it adds the class to your module. See `mk_class` for more details and examples of use (which also apply to `get_class`).

In [ ]:
#|export
def mk_class(nm, *fld_names, sup=None, doc=None, funcs=None, mod=None, **flds):
    "Create a class using `get_class` and add to the caller's module"
    if mod is None: mod = sys._getframe(1).f_locals
    res = get_class(nm, *fld_names, sup=sup, doc=doc, funcs=funcs, **flds)
    mod[nm] = res

Any `kwargs` will be added as class attributes, and `sup` is an optional (tuple of) base classes.

In [ ]:
mk_class('_t', a=1, sup=dict)
t = _t()
test_eq(t.a, 1)
assert(isinstance(t,dict))

A `__init__` is provided that sets attrs for any kwargs, and for any args (matching by position to fields), along with a `__repr__` which prints all attrs. The docstring is set to doc. You can pass `funcs` which will be added as attrs with the function names.

In [ ]:
def foo(self): return 1
mk_class('_t', 'a', sup=dict, doc='test doc', funcs=foo)

t = _t(3, b=2)
test_eq(t.a, 3)
test_eq(t.b, 2)
test_eq(t.foo(), 1)
test_eq(t.__doc__, 'test doc')
t

{}

In [ ]:
#|export
def wrap_class(nm, *fld_names, sup=None, doc=None, funcs=None, **flds):
    "Decorator: makes function a method of a new class `nm` passing parameters to `mk_class`"
    def _inner(f):
        mk_class(nm, *fld_names, sup=sup, doc=doc, funcs=listify(funcs)+[f], mod=f.__globals__, **flds)
        return f
    return _inner

In [ ]:
@wrap_class('_t', a=2)
def bar(self,x): return x+1

t = _t()
test_eq(t.a, 2)
test_eq(t.bar(3), 4)

In [ ]:
#|export
class ignore_exceptions:
    "Context manager to ignore exceptions"
    def __enter__(self): pass
    def __exit__(self, *args): 
        return True

In [ ]:
show_doc(ignore_exceptions, title_level=3)

---

[source](https://github.com/Iamalos/id-fastcore/blob/main/id_fastcore/basics.py#L133){target="_blank" style="float:right; font-size:smaller"}

### ignore_exceptions

>      ignore_exceptions ()

Context manager to ignore exceptions

In [ ]:
with ignore_exceptions(): 
    # Exception will be ignored
    raise Exception

In [ ]:
#|export
def exec_local(code, var_name):
    "Call `exec` on `code` and return the var `var_name`"
    loc = {}
    exec(code, globals(), loc)
    return loc[var_name]

In [ ]:
test_eq(exec_local("a=1", "a"), 1)

In [ ]:
exec_local("a=1", "a")

1

In [ ]:
#|export
def risinstance(types, obj=None):
    "Curried `isinstance` but with args reversed"
    types = tuplify(types)
    # return partial function that accepts only obj
    if obj is None: return partial(risinstance, types)
    # if obj is of type string, check if name of object type name (or its parent)
    if any(isinstance(t, str) for t in types):
        return any(t.__name__ in types for t in type(obj).__mro__)
    return isinstance(obj, types)

In [ ]:
type(int).__mro__

(type, object)

In [ ]:
[t.__name__ for t in type([1]).__mro__]

['list', 'object']

In [ ]:
assert risinstance(int, 1)
assert not risinstance(str, 0)
assert risinstance(int)(1)

In [ ]:
assert risinstance(('str','int'), 'a')
assert risinstance('str', 'a')
assert not risinstance('int', 'a')

In [ ]:
type('a').__mro__[0].__name__

'str'

In [ ]:
show_doc(noop)

---

### noop

>      noop (x=None, *args, **kwargs)

Do nothing

In [ ]:
noop()
test_eq(noop(1),1)

In [ ]:
show_doc(noops)

---

### noops

>      noops (x=None, *args, **kwargs)

Do nothing (method)

In [ ]:
class _t: foo = noops
test_eq(_t().foo(1),1)

# Infinite Lists

__Classes are Objects__: In Python, classes are themselves objects. They are instances of another class called metaclass. By default, in Python 3, the metaclass of any new-style class (classes inheriting from object) is type.

__Customizing Class Creation__: Metaclasses allow you to customize the way classes are created. Just as classes define how instances behave, metaclasses define how classes behave. They control the instantiation, initialization, and behavior of classes.

These lists are useful for things like padding an array or adding index column(s) to arrays. A metaclass is a class of a class. It allows you to control the creation and behavior of classes.

- `def count(self): return itertools.count()`. This method defines a property called count. When accessed on a class that uses _InfMeta as its metaclass, it returns an infinite iterator from the itertools.count() function. This iterator generates consecutive integers starting from zero (by default).

- `def zeros(self): return itertools.cycle([0])`. This method defines a property called zeros. When accessed on a class that uses _InfMeta as its metaclass, it returns an infinite iterator from the itertools.cycle() function. This iterator cycles through the provided iterable infinitely, and in this case, it cycles through the single element [0], generating an infinite stream of zeros.

- `def ones(self): return itertools.cycle([1])`. This method defines a property called ones. Similar to zeros, it returns an infinite iterator, but this time it cycles through [1], generating an infinite stream of ones.

- `def nones(self): return itertools.cycle([None])`. This method defines a property called nones. Again, it returns an infinite iterator cycling through [None], generating an infinite stream of None values.

In [ ]:
#| export
#| hide
class _InfMeta(type):
    @property
    def count(self): return itertools.count()
    @property
    def zeros(self): return itertools.cycle([0])
    @property
    def ones(self): return itertools.cycle([1])
    @property
    def nones(self): return itertools.cycle([None])

In [ ]:
#| export
class Inf(metaclass=_InfMeta):
    "Infinite lists"
    pass

In [ ]:
test_eq([o for i,o in zip(range(5), Inf.count)],
         [0, 1, 2, 3, 4])

test_eq([o for i,o in zip(range(5), Inf.zeros)],
         [0]*5)

test_eq([o for i,o in zip(range(5), Inf.ones)],
         [1]*5)

test_eq([o for i,o in zip(range(5), Inf.nones)],
        [None]*5)

## Operator Functions

In [ ]:
#|export 
_dumobj = object()
# if b is not provided returns lambda function
def _oper(op,a,b=_dumobj): return (lambda o: op(o,a)) if b is _dumobj else op(a,b)

def _mk_op(nm, mod):
    "Create an operator using `_oper` and add to the caller's module"
    # get operator.nm
    op = getattr(operator, nm)
    # internal f-n that will be added to module to be called with one or two args
    def _inner(a, b=_dumobj): return _oper(op,a,b)
    # __qualname__ provides a qualified name for classes and functions
    _inner.__name__ = _inner.__qualname__ = nm
    _inner.__doc__ = f'Same as `operator.{nm}` or returns partial if 1 arg'
    mod[nm] = _inner

In [ ]:
#| hide
_f = _oper(operator.add,1,2)
assert _f==3, 'Error'
_f = _oper(operator.add,10)
assert _f(3)==13, 'Error'

In [ ]:
#|export
def in_(x, a):
    "True if `x in a`"
    return x in a

operator.in_ = in_

In [ ]:
#|export
_all_ = ['lt','gt','le','ge','eq','ne','add','sub','mul','truediv','is_','is_not','in_', 'mod']

In [ ]:
#|export
for op in ['lt','gt','le','ge','eq','ne','add','sub','mul','truediv','is_','is_not','in_', 'mod']: _mk_op(op, globals())

In [ ]:
#|hide
globals()['lt'].__doc__

'Same as `operator.lt` or returns partial if 1 arg'

In [ ]:
# test if element is in another
assert in_('c', ('b', 'c', 'a'))
assert in_(4, [2,3,4,5])
assert in_('t', 'fastai')
test_fail(in_('h', 'fastai'))

# use in_ as a partial
# note that `fastai` here is used as a second argument in _in due to how _oper works
assert in_('fastai')('t')
assert in_([2,3,4,5])(4)
test_fail(in_('fastai')('h'))

In addition to `in_`, the following functions are provided matching the behavior of the equivalent versions in `operator`: __lt gt le ge eq ne add sub mul truediv is\_ is_not._

In [ ]:
lt(5)(3),gt(5)(3),is_(None)(None),in_([1,2])(0)

(True, False, True, False)

In [ ]:
#|export
def ret_true(*args, **kwargs):
    "Predicate: always `True`"
    return True

In [ ]:
assert ret_true(1,2,3)
assert ret_true(False)

In [ ]:
#|export
def ret_false(*args, **kwargs):
    "Predicate: always `False`"
    return False

`stop` is also used when we want to raise exception inside oneliner return. In that way direct raise will get an error 

In [ ]:
#|export
def stop(e=StopIteration):
    "Raises exception `e` (by default `StopException`)"
    raise e

In [ ]:
#|export
def gen(func, seq, cond=ret_true):
    "Like `(func(o) for o in seq if cond(func(o)))` but handles `StopIteration`"
    return itertools.takewhile(cond, map(func,seq))

In [ ]:
test_eq(gen(noop, Inf.count, lt(5)),
        range(5))
test_eq(gen(operator.neg, Inf.count, gt(-5)),
        [0,-1,-2,-3,-4])
test_eq(gen(lambda o:o if o<5 else stop(), Inf.count),
        range(5))

In [ ]:
#|export
def chunked(it, chunk_sz=None, drop_last=False, n_chunks=None):
    "Return batches from iterator `it` of size `chunk_sz` (or return `n_chunks` total)"
    # either `chunk_sz` is provided or `n_chunks`, not both
    assert bool(chunk_sz) ^ bool(n_chunks)
    if n_chunks: chunk_sz = max(math.ceil(len(it) / n_chunks), 1)
    if not isinstance(it, Iterator): it = iter(it)
    while True:
        res = list(itertools.islice(it, chunk_sz))
        if res and (len(res) == chunk_sz or not drop_last): yield res
        if len(res) < chunk_sz: return

In [ ]:
# hide
g = (i for i in range(100))
list(itertools.islice(g,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
t = list(range(10))
test_eq(chunked(t,3),      [[0,1,2], [3,4,5], [6,7,8], [9]])
test_eq(chunked(t,3,True), [[0,1,2], [3,4,5], [6,7,8],    ])

t = map(lambda o:stop() if o==6 else o, Inf.count)
test_eq(chunked(t,3), [[0, 1, 2], [3, 4, 5]])
t = map(lambda o:stop() if o==7 else o, Inf.count)
test_eq(chunked(t,3), [[0, 1, 2], [3, 4, 5], [6]])

t = np.arange(10)
test_eq(chunked(t,3),      [[0,1,2], [3,4,5], [6,7,8], [9]])
test_eq(chunked(t,3,True), [[0,1,2], [3,4,5], [6,7,8],    ])

test_eq(chunked([], 3),          [])
test_eq(chunked([], n_chunks=3), [])
     

In [ ]:
#|export
def otherwise(x, tst, y):
    "`y if tst(x) else x`"
    return y if tst(x) else x

In [ ]:
test_eq(otherwise(2+1, gt(3), 4), 3)
test_eq(otherwise(2+1, gt(2), 4), 4)     

# Attribute Helpers

These functions reduce boilerplate when setting or manipulating attributes or properties of objects.



In [ ]:
#|export
def custom_dir(c, add):
    "Implement custom `__dir__`, adding `add` to `cls`"
    return object.__dir__(c) + listify(add)

`custom_dir` allows you extract the `__dict__` property of a class and appends the list `add` to it.

In [ ]:
class _T: 
    def f(): pass

s = custom_dir(_T(), add=['foo', 'bar'])
assert {'foo', 'bar', 'f'}.issubset(s)

In [ ]:
#|export
class AttrDict(dict):
    "`dict` subclass that also provides access to keys as attrs"
    def __getattr__(self, k): return self[k] if k in self.keys() else stop(AttributeError(k))
    # choose what method to use based on the key (if it starts with _ or not)
    def __setattr__(self, k, v): (self.__setitem__, super().__setattr__)[k[0]=='_'](k,v)
    def __dir__(self): return super().__dir__() + list(self.keys())
    # to display in Jupyter Notebook
    def _repr_markdown_(self): return f'```json\n{pprint.pformat(self, indent=2)}\n```'

In [ ]:
d = AttrDict(a=1, b="two")
test_eq(d.a, 1)
test_eq(d['b'], 'two')
test_eq(d.get('c','nope'), 'nope')
d.b = 2
test_eq(d.b, 2)
test_eq(d['b'], 2)
d['b'] = 3
test_eq(d['b'], 3)
test_eq(d.b, 3)
assert 'a' in dir(d)

The `__setattr__` method in `AttrDict` is designed to differentiate between setting a dictionary item and setting an instance attribute based on the leading character of the key. If the key starts with an underscore, it treats it as a regular attribute; otherwise, it treats it as a dictionary item. This design allows for a flexible and intuitive way to interact with dictionary-like objects using attribute access syntax.

In [ ]:
#| hide
d = AttrDict()

# Setting a regular attribute
d.foo = 'bar'  # This calls self.__setitem__('foo', 'bar')
print(d)       # Output: {'foo': 'bar'}

# Setting an attribute with a leading underscore
d._private = 'secret'  # This calls super().__setattr__('_private', 'secret')
print(d._private)      # Output: 'secret'
print(d)               # Output: {'foo': 'bar'}


{'foo': 'bar'}
secret
{'foo': 'bar'}


`AttrDict` will pretty print in Jupyter Notebooks:

In [ ]:
_test_dict = {'a':1, 'b': {'c':1, 'd':2}, 'c': {'c':1, 'd':2}, 'd': {'c':1, 'd':2},
              'e': {'c':1, 'd':2}, 'f': {'c':1, 'd':2, 'e': 4, 'f':[1,2,3,4,5]}}
AttrDict(_test_dict)

```json
{ 'a': 1,
  'b': {'c': 1, 'd': 2},
  'c': {'c': 1, 'd': 2},
  'd': {'c': 1, 'd': 2},
  'e': {'c': 1, 'd': 2},
  'f': {'c': 1, 'd': 2, 'e': 4, 'f': [1, 2, 3, 4, 5]}}
```

In [ ]:
#|export
class NS(SimpleNamespace):
    "`SimpleNamespace` subclass that also adds `iter` and `dict` support"
    def __iter__(self): return iter(self.__dict__)
    def __getitem__(self, x): return self.__dict__[x]
    def __setitem__(self, x, y): self.__dict__[x] = y

In [ ]:
d = NS(**_test_dict)
d

namespace(a=1,
          b={'c': 1, 'd': 2},
          c={'c': 1, 'd': 2},
          d={'c': 1, 'd': 2},
          e={'c': 1, 'd': 2},
          f={'c': 1, 'd': 2, 'e': 4, 'f': [1, 2, 3, 4, 5]})

...but you can also index it to get/set:

In [ ]:
d['a']

1

...and iterate t:

In [ ]:
list(d)

['a', 'b', 'c', 'd', 'e', 'f']

# Collection functions

Functions that manipulate popular python collections.

In [ ]:
#| export
def partition(coll, f):
    "Partition a collection by a predicate"
    # truthes and falses
    ts, fs = [], []
    for o in coll: (fs, ts)[f(o)].append(o)
    if isinstance(coll, tuple):
        typ = type(coll)
        ts, fs = typ(ts), typ(fs)
    return ts, fs

In [ ]:
ts,fs = partition(range(10), mod(2))
test_eq(fs, [0,2,4,6,8])
test_eq(ts, [1,3,5,7,9])

In [ ]:
#|export
def flatten(o):
    "Concatenate all collections and items as a generator"
    for item in o:
        if isinstance(item, str): yield item; continue
        try: yield from flatten(item)
        except TypeError: yield item

If `item` is a string, it yields the string and uses `continue` to move to the next iteration of the loop. The `continue` statement ensures that no further processing is done on strings.
Strings are treated specially because strings are iterable (a string is a sequence of characters), and iterating over them would split them into individual characters, which is usually not desired in a flattening context.

In [ ]:
#|export
def concat(colls)->list:
    "Concatenate all collections and items as a list"
    return list(flatten(colls))

In [ ]:
concat([(o for o in range(2)),[2,3,4], 5])

[0, 1, 2, 3, 4, 5]

In [ ]:
concat([["abc", "xyz"], ["foo", "bar"]])

['abc', 'xyz', 'foo', 'bar']

In [ ]:
#|export
def strcat(its, sep:str='')->str:
    "Concatenate stringified items `its`"
    return sep.join(map(str,its))

In [ ]:
test_eq(strcat(['a',2]), 'a2')
test_eq(strcat(['a',2], ';'), 'a;2')

In [ ]:
#|export
def detuplify(x):
    "If `x` is a tuple with one thing, extract it"
    return None if len(x)==0 else x[0] if len(x)==1 and getattr(x, 'ndim', 1)==1 else x

In [ ]:
test_eq(detuplify(()),None)
test_eq(detuplify([1]),1)
test_eq(detuplify([1,2]), [1,2])
test_eq(detuplify(np.array([[1,2]])), np.array([[1,2]]))

In [ ]:
#|export
def replicate(item, match):
    "Create tuple of `item` copied `len(match)` times"
    return (item,) * len(match)

In [ ]:
t = [1,1]
test_eq(replicate([1,2], t),([1,2],[1,2]))
test_eq(replicate(1, t),(1,1))

In [ ]:
(1,3)*5

(1, 3, 1, 3, 1, 3, 1, 3, 1, 3)

In [ ]:
#|hide
#|eval: false
from nbdev import nbdev_export
nbdev_export()